In [ ]:
import os
from transformers import pipeline
import pandas as pd
import GEOparse
from transformers import AutoTokenizer
import llama 
from importlib import reload
from llama.utils import wrap_gemini_dialog  
from llama.column_description import system_message, dataset_columns_description_prompt, dataset_input





In [12]:
gse = GEOparse.get_GEO(geo="GSE1563", destdir="./data")

15-Mar-2025 10:08:36 DEBUG utils - Directory ./data already exists. Skipping.
15-Mar-2025 10:08:36 INFO GEOparse - File already exist: using local version.
15-Mar-2025 10:08:36 INFO GEOparse - Parsing ./data\GSE1563_family.soft.gz: 
15-Mar-2025 10:08:36 DEBUG GEOparse - DATABASE: GeoMiame
15-Mar-2025 10:08:36 DEBUG GEOparse - SERIES: GSE1563
15-Mar-2025 10:08:36 DEBUG GEOparse - PLATFORM: GPL8300
15-Mar-2025 10:08:40 DEBUG GEOparse - SAMPLE: GSM26805
15-Mar-2025 10:08:40 DEBUG GEOparse - SAMPLE: GSM26806
15-Mar-2025 10:08:40 DEBUG GEOparse - SAMPLE: GSM26807
15-Mar-2025 10:08:40 DEBUG GEOparse - SAMPLE: GSM26808
15-Mar-2025 10:08:40 DEBUG GEOparse - SAMPLE: GSM26809
15-Mar-2025 10:08:41 DEBUG GEOparse - SAMPLE: GSM26810
15-Mar-2025 10:08:41 DEBUG GEOparse - SAMPLE: GSM26811
15-Mar-2025 10:08:41 DEBUG GEOparse - SAMPLE: GSM26812
15-Mar-2025 10:08:41 DEBUG GEOparse - SAMPLE: GSM26813
15-Mar-2025 10:08:41 DEBUG GEOparse - SAMPLE: GSM26814
15-Mar-2025 10:08:41 DEBUG GEOparse - SAMPLE: GSM2

In [13]:
gsm_name, gsm = next(iter(gse.gsms.items()))

gsm.table

,ID_REF,VALUE,ABS_CALL,DETECTION P-VALUE
0,AFFX-MurIL2_at,211.5,A,0.50000
1,AFFX-MurIL10_at,190.2,A,0.23906
2,AFFX-MurIL4_at,96.5,A,0.68628
3,AFFX-MurFAS_at,80.8,A,0.55935
4,AFFX-BioB-5_at,847.3,P,0.00202
...,...,...,...,...
12620,160025_at,248.5,M,0.04397
12621,160027_s_at,502.8,P,0.00262
12622,160026_at,171.4,A,0.64131
12623,160043_at,183.7,P,0.00486


In [14]:
data_filename = 'GSM'
header = 4

gsm_dir = gsm.filename if hasattr(gsm, 'filename') else './data' 

data_fp = os.path.join(gsm_dir, data_filename)
description_fp = os.path.join(gsm_dir, 'description.txt')


if os.path.isfile(description_fp):
    with open(description_fp, 'r') as f:
        description = f.read()
else:
    description = ''

In [15]:
column_names = gsm.table.columns.tolist()
data_sample = gsm.table.sample(50)
data_str = data_sample.to_string()

In [16]:
reload(llama)

<module 'llama' (namespace) from ['c:\\Users\\hiwot\\Desktop\\icog\\New folder\\llama']>

In [53]:

messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': dataset_columns_description_prompt + dataset_input.format(dataset_description=description, dataset=data_str)}
]

input_prompt = wrap_gemini_dialog(messages)

In [55]:
print(input_prompt)

```json
{
  "ID_REF": {
    "description": "Unique identifier for each gene expression probe.",
    "data_type": "string"
  },
  "VALUE": {
    "description": "Numerical value representing the measured expression level of the gene.",
    "data_type": "float"
  },
  "ABS_CALL": {
    "description": "Represents the absolute call for the presence or absence of the gene, where 'A' likely stands for absent or no change, 'P' stands for present or increase, and 'M' stands for marginal change.",
    "data_type": "string"
  },
  "DETECTION P-VALUE": {
    "description": "Statistical measure of significance for the detection of the gene expression difference. Lower p-values suggest stronger evidence of differential expression.",
    "data_type": "float"
  }
}
```
